In [34]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [15]:
# distance_matrix()

import numpy as np 

pos = np.array([[2, 5],
                [3, 1],
                [1, 6],])
dim = len(pos)

D = np.zeros((dim, dim))
for i in range(dim):
    for j in range(i+1, dim):
        print(pos[i], pos[j])
        dist = np.linalg.norm(pos[i]-pos[j])**2
        D[i][j] = D[j][i] = dist

D

[2 5] [3 1]
[2 5] [1 6]
[3 1] [1 6]


array([[ 0., 17.,  2.],
       [17.,  0., 29.],
       [ 2., 29.,  0.]])

In [16]:
D = np.array([[0, 5, 5, 16],
                [5, 0, 4, 5],
                [5, 4, 0, 5],
                [16, 5, 5, 0],
                ])
dim = len(D)

In [17]:
# _I()
def _I():
    return np.identity(dim)

In [18]:
# _J()
def _J():
    return np.ones((dim, dim))

In [21]:
# Double Centering
V = _I() - (1/dim)*_J()
Z = -(1/2) * (V @ D @ V)

print(V)
print()
print(Z)

[[ 0.75 -0.25 -0.25 -0.25]
 [-0.25  0.75 -0.25 -0.25]
 [-0.25 -0.25  0.75 -0.25]
 [-0.25 -0.25 -0.25  0.75]]

[[ 4. -0. -0. -4.]
 [-0.  1. -1. -0.]
 [-0. -1.  1. -0.]
 [-4. -0. -0.  4.]]


In [22]:
# Eckart-Young decomposition
vals, U = np.linalg.eig(Z)
print(vals)
print()
print(U)
print('-'*40)
#idx = np.argsort(vals)[::-1]
idx = vals > 0
rank = np.sum(idx)
vals = vals[idx].copy()
U = (U.T[idx]).T.copy()
print(vals)
print()
print(U)
print()


Lambda = np.diag(vals)
print(Lambda)
print()

X = U @ np.sqrt(Lambda)
print(X.reshape(dim, rank))


[8. 0. 2. 0.]

[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.          0.         -0.70710678 -0.70710678]
 [ 0.          0.          0.70710678 -0.70710678]
 [-0.70710678 -0.70710678  0.          0.        ]]
----------------------------------------
[8. 2.]

[[ 0.70710678  0.        ]
 [ 0.         -0.70710678]
 [ 0.          0.70710678]
 [-0.70710678  0.        ]]

[[8. 0.]
 [0. 2.]]

[[ 2.  0.]
 [ 0. -1.]
 [ 0.  1.]
 [-2.  0.]]


In [23]:
n = len(X)
new_D = np.zeros((n, n))
for i in range(n):
    for j in range(i+1, n):
        diff = np.linalg.norm(X[i] - X[j])**2
        new_D[i][j] = new_D[j][i] = diff

new_D

array([[ 0.,  5.,  5., 16.],
       [ 5.,  0.,  4.,  5.],
       [ 5.,  4.,  0.,  5.],
       [16.,  5.,  5.,  0.]])

In [24]:
dimension = 2
X[:]

array([[ 2.,  0.],
       [ 0., -1.],
       [ 0.,  1.],
       [-2.,  0.]])

In [25]:
X.T[:dimension].T

array([[ 2.,  0.],
       [ 0., -1.],
       [ 0.,  1.],
       [-2.,  0.]])

In [26]:
n=2
X.T[:n].T

array([[ 2.,  0.],
       [ 0., -1.],
       [ 0.,  1.],
       [-2.,  0.]])

In [27]:
class DistanceMatrix:
    def __init__(self, pos):
        self.pos = pos
        self.check_is_Eucledian_distance()

    
    def distance_matrix(self):
        n = len(self.pos)
        D = np.zeros((n, n))
        for i in range(n):
            for j in range(i+1, n):
                diff = np.linalg.norm(self.pos[i] - self.pos[j])**2
                D[i][j] = D[j][i] = diff
        return D


    def check_is_Eucledian_distance(self):
        n = len(self.pos)
        D = self.distance_matrix()
        d = np.sqrt(D)

        for i in range(n):
            for j in range(i+1, n):
                for k in range(n):
                    if (k!=i) and (k!=j):
                        if d[i][j]+d[i][k]<d[j][k]:
                            raise ValueError(f"D is not Eucledian distance matrix")
        return True

In [28]:
DistanceMatrix(X.T[:n].T).distance_matrix()

array([[ 0.,  5.,  5., 16.],
       [ 5.,  0.,  4.,  5.],
       [ 5.,  4.,  0.,  5.],
       [16.,  5.,  5.,  0.]])

修正後

In [29]:
X = np.array([[2, 5],
              [3, 1],
              [1, 6],])

G = X@X.T
diag = np.diag(G)
D2 = diag[:, None] + diag[None, :] - 2*G
D2[D2<0] = np.maximum(D2[D2<0], 0.0)
np.fill_diagonal(D2, 0.0)
D2

array([[ 0, 17,  2],
       [17,  0, 29],
       [ 2, 29,  0]])

In [30]:
tol = 1e-10
A = D2
A = 0.5*(A+A.T)
vals = np.linalg.eigvalsh(A)
np.min(vals) >= -tol

False

In [33]:
import pandas as pd
import numpy as np
from modules.MDS import squared_distance_matrix

data = [{"id": 1, "data_A": 1, "data_B": 3},
        {"id": 2, "data_A": 3, "data_B": 2},
        {"id": 3, "data_A": 3, "data_B": 4},
        {"id": 4, "data_A": 2, "data_B": 2},
        ]

df = pd.DataFrame(data)
pos = np.asarray(df[["data_A", "data_B"]])
squared_distance_matrix(pos)

array([[0., 5., 5., 2.],
       [5., 0., 4., 1.],
       [5., 4., 0., 5.],
       [2., 1., 5., 0.]])